# Exercise 6

## SVM & Regularization


For this homework we consider a set of observations on a number of red and white wine varieties involving their chemical properties and ranking by tasters. Wine industry shows a recent growth spurt as social drinking is on the rise. The price of wine depends on a rather abstract concept of wine appreciation by wine tasters, opinion among whom may have a high degree of variability. Pricing of wine depends on such a volatile factor to some extent. Another key factor in wine certification and quality assessment is physicochemical tests which are laboratory-based and takes into account factors like acidity, pH level, presence of sugar and other chemical properties. For the wine market, it would be of interest if human quality of tasting can be related to the chemical properties of wine so that certification and quality assessment and assurance process is more controlled.

Two datasets are available of which one dataset is on red wine and have 1599 different varieties and the other is on white wine and have 4898 varieties. All wines are produced in a particular area of Portugal. Data are collected on 12 different properties of the wines one of which is Quality, based on sensory data, and the rest are on chemical properties of the wines including density, acidity, alcohol content etc. All chemical properties of wines are continuous variables. Quality is an ordinal variable with possible ranking from 1 (worst) to 10 (best). Each variety of wine is tasted by three independent tasters and the final rank assigned is the median rank given by the tasters.

A predictive model developed on this data is expected to provide guidance to vineyards regarding quality and price expected on their produce without heavy reliance on volatility of wine tasters.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [3]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
1041,7.7,0.275,0.30,1.00,0.039,19.0,75.0,0.99200,3.01,0.56,10.7,5,white
2754,5.9,0.170,0.28,0.70,0.027,5.0,28.0,0.98985,3.13,0.32,10.6,5,white
3879,6.8,0.730,0.20,6.60,0.054,25.0,65.0,0.99324,3.12,0.28,11.1,4,white
4881,5.0,0.235,0.27,11.75,0.030,34.0,118.0,0.99540,3.07,0.50,9.4,6,white
4072,7.5,0.160,0.38,12.70,0.043,70.5,163.0,0.99706,3.15,0.82,10.4,7,white


# Exercise 6.1

Show the frecuency table of the quality by type of wine

In [4]:
t4 = pd.pivot_table(data, index = ['type'], values = 'quality', aggfunc = 'count')

t4

,quality
type,
red,1599
white,4898


In [5]:
my_tab = pd.crosstab(index=data["type"], columns = data['quality'],margins = True)
my_tab

quality,3,4,5,6,7,8,9,All
type,,,,,,,,
red,10,53,681,638,199,18,0,1599
white,20,163,1457,2198,880,175,5,4898
All,30,216,2138,2836,1079,193,5,6497


In [6]:
data['q_2'] = data.quality.map({3:0,4:0,5:0,6:1,7:1,8:1,9:1})
data.head()

#data['CH'] = data.Churn.map({'False.':0, 'True.':1})

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type,q_2
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,white,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,white,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,white,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white,1


In [7]:
datared = data[data['type'] == 'red']
datawhite = data[data['type'] == 'white']
datared.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type,q_2
4898,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red,0
4899,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red,0
4900,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red,0
4901,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red,1
4902,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red,0


# SVM

# Exercise 6.2

* Standarized the features (not the quality)
* Create a binary target for each type of wine
* Create two Linear SVM's for the white and red wines, repectively.


In [8]:
data_nm = data[['type', 'quality','q_2']]

In [9]:
import warnings
warnings.filterwarnings('ignore') 


var=list(data)
var.remove("quality")
var.remove("type")
var.remove("q_2")


for i in range(0,len(var)): 
    data_nm[var[i]+"_1"] = (data[var[i]]-data[var[i]].mean())/(data[var[i]].std(ddof=0)) 
data_nm.head()

,type,quality,q_2,fixed acidity_1,volatile acidity_1,citric acid_1,residual sugar_1,chlorides_1,free sulfur dioxide_1,total sulfur dioxide_1,density_1,pH_1,sulphates_1,alcohol_1
0,white,6,1,-0.166089,-0.423183,0.284686,3.206929,-0.314975,0.815565,0.959976,2.102214,-1.359049,-0.546178,-1.418558
1,white,6,1,-0.706073,-0.240949,0.147046,-0.807837,-0.200790,-0.931107,0.287618,-0.232332,0.506915,-0.277351,-0.831615
2,white,6,1,0.682458,-0.362438,0.559966,0.306208,-0.172244,-0.029599,-0.331660,0.134525,0.258120,-0.613385,-0.328521
3,white,6,1,-0.011808,-0.666161,0.009406,0.642523,0.056126,0.928254,1.243074,0.301278,-0.177272,-0.882212,-0.496219
4,white,6,1,-0.011808,-0.666161,0.009406,0.642523,0.056126,0.928254,1.243074,0.301278,-0.177272,-0.882212,-0.496219


In [10]:
from sklearn.svm import SVC # "Support Vector Classifier"
from sklearn.model_selection import train_test_split

In [11]:
data_R=data_nm[(data_nm["type"]=='red')]
data_W=data_nm[(data_nm["type"]=='white')]

data_R_2=data_R.drop("type",axis=1)
data_R_2=data_R_2.drop("quality",axis=1)

data_W_2=data_W.drop("type",axis=1)
data_W_2=data_W_2.drop("quality",axis=1)



In [12]:
y_r = data_R_2["q_2"].values
X_r = data_R_2[['fixed acidity_1', 'volatile acidity_1','citric acid_1','residual sugar_1','chlorides_1','free sulfur dioxide_1','total sulfur dioxide_1','density_1','pH_1','sulphates_1','alcohol_1']].values

In [13]:
validation = 0.30
seed = 8
X_train_rw, X_test_rw= train_test_split(X_r,test_size=validation, random_state=seed)
y_train_rw, y_test_rw = train_test_split(y_r,test_size=validation, random_state=seed)

In [14]:
clf_rw = SVC(kernel='linear')
clf_rw.fit(X_train_rw, y_train_rw)
clf_rw.score(X_test_rw,y_test_rw)

0.7458333333333333

In [15]:
y_w = data_W_2["q_2"].values
X_w = data_W_2[['fixed acidity_1', 'volatile acidity_1','citric acid_1','residual sugar_1','chlorides_1','free sulfur dioxide_1','total sulfur dioxide_1','density_1','pH_1','sulphates_1','alcohol_1']].values

In [16]:
X_train_ww, X_test_ww= train_test_split(X_w,test_size=validation, random_state=seed)
y_train_ww, y_test_ww = train_test_split(y_w,test_size=validation, random_state=seed)

In [17]:
clf_ww = SVC(kernel='linear')
clf_ww.fit(X_train_ww, y_train_ww)
clf_ww.decision_function(X_test_ww)

array([-0.29967265,  1.42923183, -0.54489265, ..., -0.37978594,
        2.13360039, -0.76210182])

In [18]:
clf_ww.predict(X_test_ww)

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

In [19]:
clf_ww.score(X_test_ww,y_test_ww)

0.7598639455782313

In [20]:
d = {'Result': ['red wine', 'white wine'],'Accuracy': [clf_rw.score(X_test_rw,y_test_rw),clf_ww.score(X_test_ww,y_test_ww)],}
result= pd.DataFrame(data=d)
result

,Result,Accuracy
0,red wine,0.745833
1,white wine,0.759864


# Exercise 6.3

Test the two SVM's using the different kernels (‘poly’, ‘rbf’, ‘sigmoid’)


In [21]:
pol_r = SVC(kernel='poly')
pol_w= SVC(kernel='poly')

rbf_r= SVC(kernel='rbf')
rbf_w= SVC(kernel='rbf')

sig_r= SVC(kernel='sigmoid')
sig_w= SVC(kernel='sigmoid')

In [22]:
pol_r.fit(X_train_rw, y_train_rw)
pol_w.fit(X_train_ww, y_train_ww)

rbf_r.fit(X_train_rw, y_train_rw)
rbf_w.fit(X_train_ww, y_train_ww)

sig_r.fit(X_train_rw, y_train_rw)
sig_w.fit(X_train_ww, y_train_ww)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='sigmoid', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [23]:
result_2 = {'Result_2': ['red wine', 'red wine','red wine','red wine','white wine', 'white wine','white wine','white wine'],'Kernel': ['linear','poly', 'rbf','sigmoid','linear','poly', 'rbf','sigmoid'],'Accuracy': [clf_rw.score(X_test_rw,y_test_rw),pol_r.score(X_test_rw, y_test_rw),rbf_r.score(X_test_rw, y_test_rw),sig_r.score(X_test_rw, y_test_rw),clf_ww.score(X_test_ww,y_test_ww),pol_w.score(X_test_ww, y_test_ww),rbf_w.score(X_test_ww, y_test_ww),sig_w.score(X_test_ww, y_test_ww)],}
result_3 = pd.DataFrame(data=result_2)
pd.pivot_table(result_3,index = ["Result_2"], values=["Accuracy"],columns=["Kernel"],aggfunc=[np.sum])

sum                              
            Accuracy                              
Kernel        linear      poly       rbf   sigmoid
Result_2                                          
red wine    0.745833  0.768750  0.775000  0.608333
white wine  0.759864  0.740136  0.767347  0.673469

Con base en la tabla anterior, podemos identificar que el kernel con mejor desempeño es `rbf` y el de peor desempeño `sigmoid`

# Exercise 6.4
Using the best SVM find the parameters that gives the best performance

'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]

In [24]:
C = [0.1, 1, 10, 100, 1000]
gamma = [0.01, 0.001, 0.0001]

In [25]:
df_r = pd.DataFrame(0, index=C, columns=gamma)
df_w = pd.DataFrame(0, index=C, columns=gamma)

In [26]:
for i in range(0,len(C)): 
    for j in range(0,len(gamma)):
        rbf_rw1 = SVC(kernel='rbf',C=C[i],gamma=gamma[j]).fit(X_train_rw, y_train_rw)
        df_r.iloc[i,j]=rbf_rw1.score(X_test_rw, y_test_rw)

In [27]:
print ( "\033[1;37mAccuracy Red Wine (RBF)")

df_r

Accuracy Red Wine (RBF)


,0.01,0.001,0.0001
0.1,0.712500,0.516667,0.516667
1.0,0.750000,0.716667,0.516667
10.0,0.760417,0.747917,0.718750
100.0,0.766667,0.750000,0.743750
1000.0,0.772917,0.747917,0.747917


In [28]:
for i in range(0,len(C)): 
    for j in range(0,len(gamma)):
        rbf_ww1 = SVC(kernel='rbf',C=C[i],gamma=gamma[j]).fit(X_train_ww, y_train_ww)
        df_w.iloc[i,j]=rbf_ww1.score(X_test_ww, y_test_ww)

In [29]:
print ( "\033[1;37mAccuracy Withe Wine (RBF)")
df_w

Accuracy Withe Wine (RBF)


,0.01,0.001,0.0001
0.1,0.689116,0.665986,0.665986
1.0,0.759864,0.689116,0.665986
10.0,0.770748,0.758503,0.689116
100.0,0.773469,0.767347,0.760544
1000.0,0.763946,0.770748,0.761905


# Exercise 6.5

Compare the results with other methods

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [31]:
lm = linear_model.LogisticRegression()
clflrr=lm.fit(X_train_rw, y_train_rw)
clflrw=lm.fit(X_train_ww, y_train_ww)

In [40]:
DF=data_R_2.drop("q_2",axis=1)

In [45]:
a=clflrr.coef_
a=pd.DataFrame(a.reshape(-1,1))
a.rename(columns={0:"Coef"}, inplace=True)
a["Features"]=DF.columns
print("\033[1;37mCoef Red Wine Logit")
a

Coef Red Wine Logit


,Coef,Features
0,0.150817,fixed acidity_1
1,-1.058588,volatile acidity_1
2,0.070306,citric acid_1
3,0.891986,residual sugar_1
4,0.014499,chlorides_1
5,0.215898,free sulfur dioxide_1
6,-0.067902,total sulfur dioxide_1
7,-1.058929,density_1
8,0.231247,pH_1
9,0.285077,sulphates_1


In [47]:
b=clflrw.coef_
b=pd.DataFrame(b.reshape(-1,1))
b.rename(columns={0:"Coef"}, inplace=True)
b["Features"]=DF.columns
print("\033[1;37mCoef White Wine Logit")
b

Coef White Wine Logit


,Coef,Features
0,0.150817,fixed acidity_1
1,-1.058588,volatile acidity_1
2,0.070306,citric acid_1
3,0.891986,residual sugar_1
4,0.014499,chlorides_1
5,0.215898,free sulfur dioxide_1
6,-0.067902,total sulfur dioxide_1
7,-1.058929,density_1
8,0.231247,pH_1
9,0.285077,sulphates_1


In [48]:
print("\033[1;37mAccuracy Red Wine Logit: ", clflrr.score(X_test_rw, y_test_rw))

Accuracy Red Wine Logit:  0.6125


In [49]:
print("\033[1;37mAccuracy White Wine Logit: ", clflrw.score(X_test_ww, y_test_ww)

SyntaxError: unexpected EOF while parsing (<ipython-input-49-b8c780eddc7d>, line 1)

# Regularization

# Exercise 6.6


* Train a linear regression to predict wine quality (Continous)

* Analyze the coefficients

* Evaluate the RMSE

In [50]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import SGDRegressor
from sklearn import metrics
from sklearn.linear_model import LinearRegression

In [51]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [52]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
5787,10.7,0.90,0.34,6.6,0.112,23.0,99.0,1.00289,3.22,0.68,9.3,5,red
2094,6.6,0.22,0.53,15.1,0.052,22.0,136.0,0.99860,2.94,0.35,9.4,5,white
1244,6.4,0.26,0.22,5.1,0.037,23.0,131.0,0.99440,3.29,0.32,10.1,5,white
391,6.5,0.36,0.32,1.1,0.031,13.0,66.0,0.99160,3.10,0.46,10.6,5,white
123,6.9,0.19,0.28,5.0,0.058,14.0,146.0,0.99520,3.29,0.36,9.1,6,white


In [54]:
data_3 = data[['type']]

var=list(data)
var.remove("type")
for i in range(0,len(var)): 
    data_3[var[i]+"_1"] = (data[var[i]]-data[var[i]].mean())/(data[var[i]].std(ddof=0)) 
data_3.sample(5)

,type,fixed acidity_1,volatile acidity_1,citric acid_1,residual sugar_1,chlorides_1,free sulfur dioxide_1,total sulfur dioxide_1,density_1,pH_1,sulphates_1,alcohol_1,quality_1
2119,white,0.605317,-1.091373,0.284686,-0.828857,-0.657530,-0.931107,0.110681,-0.198981,-0.115073,-0.949419,-0.160823,-2.082458
3106,white,0.296754,-0.423183,1.385805,-0.471522,-0.372068,-0.142287,0.641490,-1.136135,-1.234651,-0.008524,0.761516,0.207999
3169,white,-1.168917,-1.091373,-0.403514,0.180090,-0.372068,-0.142287,-0.048561,-0.365735,-0.985856,0.327510,-0.496219,0.207999
1462,white,0.142473,-1.091373,1.179346,-0.891916,-0.029513,-0.705730,0.606103,-0.999397,0.071523,-0.411765,0.593818,0.207999
6171,red,0.219614,1.459898,-0.816433,-0.723758,0.484319,0.195778,-1.269422,0.081164,-0.737061,-0.680592,-0.999313,-0.937230


In [55]:
data_R2=data_3[(data_3["type"]=='red')]
data_W2=data_3[(data_3["type"]=='white')]

data_R2_2=data_R2.drop("type",axis=1)
data_W2_2=data_W2.drop("type",axis=1)

y_r = data_R2_2["quality_1"].values
X_r = data_R2_2[['fixed acidity_1', 'volatile acidity_1','citric acid_1','residual sugar_1','chlorides_1','free sulfur dioxide_1','total sulfur dioxide_1','density_1','pH_1','sulphates_1','alcohol_1']].values

y_w = data_W2_2["quality_1"].values
X_w = data_W2_2[['fixed acidity_1', 'volatile acidity_1','citric acid_1','residual sugar_1','chlorides_1','free sulfur dioxide_1','total sulfur dioxide_1','density_1','pH_1','sulphates_1','alcohol_1']].values

In [58]:
X_train_rw, X_test_rw= train_test_split(X_r,test_size=validation, random_state=seed)
y_train_rw, y_test_rw = train_test_split(y_r,test_size=validation, random_state=seed)

X_train_ww, X_test_ww= train_test_split(X_w,test_size=validation, random_state=seed)
y_train_ww, y_test_ww = train_test_split(y_w,test_size=validation, random_state=seed)

In [60]:
lm_rw=LinearRegression()
lm_ww=LinearRegression()

lm_rw.fit(X_train_rw, y_train_rw)
lm_ww.fit(X_train_ww, y_train_ww)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [61]:
DF=data_R2_2.drop("quality_1",axis=1)

In [62]:
result_rw=pd.DataFrame({"Features":DF.columns,"Coef":lm_rw.coef_})
result_rw.loc[len(result_rw)]=["Intercept",lm_rw.intercept_]
print("\033[1;37mCoef red Wine")
result_rw

Coef red Wine


,Features,Coef
0,fixed acidity_1,0.034622
1,volatile acidity_1,-0.189044
2,citric acid_1,-0.013334
3,residual sugar_1,0.070546
4,chlorides_1,-0.056856
5,free sulfur dioxide_1,0.117073
6,total sulfur dioxide_1,-0.222388
7,density_1,-0.102047
8,pH_1,-0.079498
9,sulphates_1,0.171966


De acuerdo con el resultado anterior, las variables con mayor impacto para los vinos rojos son: alcohol y dióxido de sulfuro

In [63]:
result_ww=pd.DataFrame({"Features":DF.columns,"Coef":lm_ww.coef_})
result_ww.loc[len(result_ww)]=["Intercept",lm_ww.intercept_]
print("\033[1;37mCoef White Wine")
result_ww

Coef White Wine


,Features,Coef
0,fixed acidity_1,0.162301
1,volatile acidity_1,-0.353241
2,citric acid_1,0.002612
3,residual sugar_1,0.528768
4,chlorides_1,-0.005499
5,free sulfur dioxide_1,0.079631
6,total sulfur dioxide_1,0.011639
7,density_1,-0.714809
8,pH_1,0.169781
9,sulphates_1,0.119537


De acuerdo con el resultado anterior, las variables con mayor impacto para los vinos blancos son: azúcar residual y la densidad

In [65]:
y_pred_rw=lm_rw.predict(X_test_rw)
y_pred_ww=lm_ww.predict(X_test_ww)

print("\033[1;38mRMSE RW:",np.sqrt(metrics.mean_squared_error(y_test_rw, y_pred_rw)))
print("\033[1;38mRMSE WW:",np.sqrt(metrics.mean_squared_error(y_test_ww, y_pred_ww)))

RMSE RW: 0.7347069875091722
RMSE WW: 0.8602813759403796


# Exercise 6.7

* Estimate a ridge regression with alpha equals 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

In [66]:
from sklearn.linear_model import Ridge
ridgereg_rw = Ridge(alpha=0.1, normalize=True)
ridgereg_ww = Ridge(alpha=0.1, normalize=True)

ridgereg_rw.fit(X_train_rw, y_train_rw)
ridgereg_ww.fit(X_train_ww, y_train_ww)

y_pred_rw= ridgereg_rw.predict(X_test_rw)
y_pred_ww= ridgereg_ww.predict(X_test_ww)

print("\033[1;38mRMSE RW:",np.sqrt(metrics.mean_squared_error(y_test_rw, y_pred_rw)))
print("\033[1;38mRMSE WW:",np.sqrt(metrics.mean_squared_error(y_test_ww, y_pred_ww)))

RMSE RW: 0.7361027118013673
RMSE WW: 0.8599941688694003


In [67]:
result_rw=pd.DataFrame({"Features":DF.columns,"Coef":ridgereg_rw.coef_})
result_rw.loc[len(result_rw)]=["Intercept",ridgereg_rw.intercept_]
print("\033[1;38mCoef Red Wine alpha=0.1")
result_rw

Coef Red Wine alpha=0.1


,Features,Coef
0,fixed acidity_1,0.041464
1,volatile acidity_1,-0.170797
2,citric acid_1,0.010278
3,residual sugar_1,0.081368
4,chlorides_1,-0.053532
5,free sulfur dioxide_1,0.088178
6,total sulfur dioxide_1,-0.193391
7,density_1,-0.138735
8,pH_1,-0.053309
9,sulphates_1,0.160257


In [69]:
result_rw=pd.DataFrame({"Features":DF.columns,"Coef":ridgereg_ww.coef_})
result_rw.loc[len(result_rw)]=["Intercept",ridgereg_ww.intercept_]
print("\033[1;38mCoef White Wine alpha = 0.1")
result_rw

Coef White Wine alpha = 0.1


,Features,Coef
0,fixed acidity_1,-0.026207
1,volatile acidity_1,-0.317153
2,citric acid_1,0.004017
3,residual sugar_1,0.181052
4,chlorides_1,-0.060570
5,free sulfur dioxide_1,0.093984
6,total sulfur dioxide_1,-0.037827
7,density_1,-0.182394
8,pH_1,0.066188
9,sulphates_1,0.075046


In [70]:
ridgereg_rw = Ridge(alpha=1, normalize=True)
ridgereg_ww = Ridge(alpha=1, normalize=True)

ridgereg_rw.fit(X_train_rw, y_train_rw)
ridgereg_ww.fit(X_train_ww, y_train_ww)

y_pred_rw= ridgereg_rw.predict(X_test_rw)
y_pred_ww= ridgereg_ww.predict(X_test_ww)

print("\033[1;38mRMSE RW:",np.sqrt(metrics.mean_squared_error(y_test_rw, y_pred_rw)))
print("\033[1;38mRMSE WW:",np.sqrt(metrics.mean_squared_error(y_test_ww, y_pred_ww)))

RMSE RW: 0.7637949754977287
RMSE WW: 0.902661279376848


In [71]:
result_ww=pd.DataFrame({"Features":x3.columns,"Coef":ridgereg_rw.coef_})
result_ww.loc[len(result_ww)]=["Intercept",ridgereg_rw.intercept_]
print("\033[1;38mCoef Red Wine alpha=1")
result_ww

Coef Red Wine alpha=1


,Features,Coef
0,fixed acidity_1,0.024724
1,volatile acidity_1,-0.112632
2,citric acid_1,0.039858
3,residual sugar_1,0.035309
4,chlorides_1,-0.032391
5,free sulfur dioxide_1,0.016611
6,total sulfur dioxide_1,-0.100730
7,density_1,-0.111194
8,pH_1,-0.020757
9,sulphates_1,0.095388


In [72]:
result_ww=pd.DataFrame({"Features":x3.columns,"Coef":ridgereg_ww.coef_})
result_ww.loc[len(result_ww)]=["Intercept",ridgereg_ww.intercept_]
print("\033[1;38mCoef White Wine alpha=1")
result_ww

Coef White Wine alpha=1


,Features,Coef
0,fixed acidity_1,-0.047600
1,volatile acidity_1,-0.161647
2,citric acid_1,0.006394
3,residual sugar_1,0.030357
4,chlorides_1,-0.091403
5,free sulfur dioxide_1,0.048169
6,total sulfur dioxide_1,-0.044526
7,density_1,-0.090660
8,pH_1,0.034547
9,sulphates_1,0.038497


# Exercise 6.8

* Estimate a lasso regression with alpha equals 0.01, 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

In [74]:
from sklearn.linear_model import Lasso
lassoreg_rw = Lasso(alpha=0.01, normalize=True)
lassoreg_ww = Lasso(alpha=0.01, normalize=True)

lassoreg_rw.fit(X_train_rw, y_train_rw)
lassoreg_ww.fit(X_train_ww, y_train_ww)

y_pred_rw= lassoreg_rw.predict(X_test_rw)
y_pred_ww= lassoreg_ww.predict(X_test_ww)

print("\033[1;38mRMSE RW:",np.sqrt(metrics.mean_squared_error(y_test_rw, y_pred_rw)))
print("\033[1;38mRMSE WW:",np.sqrt(metrics.mean_squared_error(y_test_ww, y_pred_ww)))

RMSE RW: 0.8571516451515133
RMSE WW: 1.009511790345014


In [75]:
result_rw=pd.DataFrame({"Features":DF.columns,"Coef":lassoreg_rw.coef_})
result_rw.loc[len(result_ww)]=["Intercept",lassoreg_rw.intercept_]
print("\033[1;38mCoef Red Wine alpha=0.01")
result_rw

Coef Red Wine alpha=0.01


,Features,Coef
0,fixed acidity_1,0.000000
1,volatile acidity_1,-0.006221
2,citric acid_1,0.000000
3,residual sugar_1,0.000000
4,chlorides_1,-0.000000
5,free sulfur dioxide_1,-0.000000
6,total sulfur dioxide_1,-0.000000
7,density_1,-0.000000
8,pH_1,-0.000000
9,sulphates_1,0.000000


In [76]:
result_ww=pd.DataFrame({"Features":x3.columns,"Coef":lassoreg_ww.coef_})
result_ww.loc[len(result_ww)]=["Intercept",lassoreg_ww.intercept_]
print("\033[1;38mCoef White Wine alpha=0.01")
result_ww

Coef White Wine alpha=0.01


,Features,Coef
0,fixed acidity_1,-0.000000
1,volatile acidity_1,-0.000000
2,citric acid_1,-0.000000
3,residual sugar_1,-0.000000
4,chlorides_1,-0.000000
5,free sulfur dioxide_1,0.000000
6,total sulfur dioxide_1,-0.000000
7,density_1,-0.000000
8,pH_1,0.000000
9,sulphates_1,0.000000


In [77]:
lassoreg_rw = Lasso(alpha=0.1, normalize=True)
lassoreg_ww = Lasso(alpha=0.1, normalize=True)

lassoreg_rw.fit(X_train_rw, y_train_rw)
lassoreg_ww.fit(X_train_ww, y_train_ww)

y_pred_rw= lassoreg_rw.predict(X_test_rw)
y_pred_ww= lassoreg_ww.predict(X_test_ww)

print("\033[1;38mRMSE RW:",np.sqrt(metrics.mean_squared_error(y_test_rw, y_pred_rw)))
print("\033[1;38mRMSE WW:",np.sqrt(metrics.mean_squared_error(y_test_ww, y_pred_ww)))

RMSE RW: 0.9046773155826463
RMSE WW: 1.009511790345014


In [78]:
result_rw=pd.DataFrame({"Features":DF.columns,"Coef":lassoreg_rw.coef_})
result_rw.loc[len(result_ww)]=["Intercept",lassoreg_rw.intercept_]
print("\033[1;38mCoef Red Wine alpha=0.1")
result_rw

Coef Red Wine alpha=0.1


,Features,Coef
0,fixed acidity_1,0.000000
1,volatile acidity_1,-0.000000
2,citric acid_1,0.000000
3,residual sugar_1,0.000000
4,chlorides_1,-0.000000
5,free sulfur dioxide_1,-0.000000
6,total sulfur dioxide_1,-0.000000
7,density_1,-0.000000
8,pH_1,-0.000000
9,sulphates_1,0.000000


In [79]:
result_ww=pd.DataFrame({"Features":DF.columns,"Coef":lassoreg_ww.coef_})
result_ww.loc[len(result_ww)]=["Intercept",lassoreg_ww.intercept_]
print("\033[1;38mCoef White Wine alpha=0.1")
result_ww

Coef White Wine alpha=0.1


,Features,Coef
0,fixed acidity_1,-0.000000
1,volatile acidity_1,-0.000000
2,citric acid_1,-0.000000
3,residual sugar_1,-0.000000
4,chlorides_1,-0.000000
5,free sulfur dioxide_1,0.000000
6,total sulfur dioxide_1,-0.000000
7,density_1,-0.000000
8,pH_1,0.000000
9,sulphates_1,0.000000


# Exercise 6.9

* Create a binary target

* Train a logistic regression to predict wine quality (binary)

* Analyze the coefficients

* Evaluate the f1score

# Exercise 6.10

* Estimate a regularized logistic regression using:
* C = 0.01, 0.1 & 1.0
* penalty = ['l1, 'l2']
* Compare the coefficients and the f1score